### Capstone 3: Data Wrangling - Snowflake Cloud Datawarehouse vs. On-Prem MSSQL Comparison

#### Import Relevant Packages

In [37]:
import pandas as pd 
import sqlalchemy 
import matplotlib.pyplot as plt 
import pyodbc
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
import os 
import warnings


warnings.filterwarnings("ignore")

#### Connect to Local MSSQL Instance and Load SQL Query

In [38]:
# Define connection string
connection_string = 'mssql+pyodbc://REHAN-DESKTOP\SQLEXPRESS/AdventureWorks2022?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'


# Create SQLAlchemy Engine
engine = sqlalchemy.create_engine(connection_string)

# SQL query
query = """ SELECT
    SOH.SalesOrderID,
    SOH.OrderDate,
    P.ProductID,
    P.Name AS ProductName,
    SOD.OrderQty,
    SOD.UnitPrice,
    SOD.LineTotal
FROM
Sales.SalesOrderHeader AS SOH
JOIN Sales.SalesOrderDetail AS SOD ON SOH.SalesOrderID = SOD.SalesOrderID
JOIN Sales.Customer AS C ON SOH.CustomerID = C.CustomerID
JOIN Production.Product AS P ON SOD.ProductID = P.ProductID
ORDER BY SOH.OrderDate DESC """


#### Connect to Snowflake Cloud Instance

In [39]:
# Add Connection Parameters
snowflake_conn = snowflake.connector.connect(
    user='REHANK1990',
    password='Bismillah!1',
    account='sajugab-fma81222',
    warehouse='COMPUTE_WH',
    database='ADVENTUREWORKS2022',
    schema='ADVENTUREWORKS2022',
    role='ACCOUNTADMIN'
)

    
snowflake_query="""SELECT
    SOH.SalesOrderID,
    SOH.OrderDate,
    P.ProductID,
    P.Name AS ProductName,
    SOD.OrderQty,
    SOD.UnitPrice,
    SOD.LineTotal
FROM "Sales.SalesOrderHeader" AS SOH
JOIN "Sales.SalesOrderDetail" AS SOD ON SOH.SalesOrderID = SOD.SalesOrderID
JOIN "Sales.Customer" AS C ON SOH.CustomerID = C.CustomerID
JOIN "Production.Product" AS P ON SOD.ProductID = P.ProductID
ORDER BY SOH.OrderDate DESC"""


#### Add Both Queries Into Dataframe

###### MSSQL Dataframe


In [40]:
# Execute query and read data into DataFrame (MSSQL)
sql_df = pd.read_sql(query, engine)

# Perform data analysis or manipulation
print(sql_df.head())  # Display first few rows of DataFrame

#capture df load time
sql_load_seconds = 7.8

   SalesOrderID  OrderDate  ProductID            ProductName  OrderQty  \
0         75084 2014-06-30        876    Hitch Rack - 4-Bike         1   
1         75085 2014-06-30        712           AWC Logo Cap         1   
2         75085 2014-06-30        877  Bike Wash - Dissolver         1   
3         75086 2014-06-30        877  Bike Wash - Dissolver         1   
4         75087 2014-06-30        707  Sport-100 Helmet, Red         1   

   UnitPrice  LineTotal  
0     120.00     120.00  
1       8.99       8.99  
2       7.95       7.95  
3       7.95       7.95  
4      34.99      34.99  


###### Snowflake Dataframe


In [41]:
# Create a DataFrame using fetched rows and column names
snowflake_df = pd.read_sql(snowflake_query, snowflake_conn)

# Close the cursor and connection when done
#cursor.close()
#snowflake_conn.close()

# Now you can work with the DataFrame
print(snowflake_df.head(10))  # Display the first few rows of the DataFrame

#Snowflake Load Time 
snowflake_load_seconds = 0.6

   SALESORDERID  ORDERDATE  PRODUCTID                      PRODUCTNAME  \
0         75119 2014-06-30        873              Patch Kit/8 Patches   
1         75123 2014-06-30        712                     AWC Logo Cap   
2         75123 2014-06-30        879           All-Purpose Bike Stand   
3         75123 2014-06-30        878            Fender Set - Mountain   
4         75122 2014-06-30        712                     AWC Logo Cap   
5         75122 2014-06-30        878            Fender Set - Mountain   
6         75121 2014-06-30        707            Sport-100 Helmet, Red   
7         75121 2014-06-30        930                 HL Mountain Tire   
8         75120 2014-06-30        712                     AWC Logo Cap   
9         75120 2014-06-30        884  Short-Sleeve Classic Jersey, XL   

   ORDERQTY  UNITPRICE  LINETOTAL  
0         1          2          2  
1         1          9          9  
2         1        159        159  
3         1         22         22  
4    

#### Set Column Names to Match

In [42]:
#set snowflake df columns to be the same as sql df
snowflake_df.columns = sql_df.columns 
print(sql_df.columns)

Index(['SalesOrderID', 'OrderDate', 'ProductID', 'ProductName', 'OrderQty',
       'UnitPrice', 'LineTotal'],
      dtype='object')


#### Check Data Types in Both Dataframes

In [43]:
#check datatypes for sql df
sql_df.dtypes

SalesOrderID             int64
OrderDate       datetime64[ns]
ProductID                int64
ProductName             object
OrderQty                 int64
UnitPrice              float64
LineTotal              float64
dtype: object

In [44]:
#check datatypes for snowflake df
snowflake_df.dtypes

SalesOrderID             int64
OrderDate       datetime64[ns]
ProductID                int64
ProductName             object
OrderQty                 int64
UnitPrice                int64
LineTotal                int64
dtype: object

In [30]:
#change snowflake dtypes to match sql_df
snowflake_df[['UnitPrice', 'LineTotal']] = snowflake_df[['UnitPrice', 'LineTotal']].astype('int64')

,UnitPrice,LineTotal
0,120,120
1,8,8
2,7,7
3,7,7
4,34,34
...,...,...
121312,3578,3578
121313,3399,3399
121314,3399,3399
121315,699,699


SalesOrderID             int64
OrderDate       datetime64[ns]
ProductID                int64
ProductName             object
OrderQty                 int64
UnitPrice              float64
LineTotal              float64
dtype: object